- Add coordinates of the Checkout and return kiosk
     - Get coordinates from heartland_bikeshare
     - add columns for coordinates of the Checkout and return kiosk 

   **Important** Some of the kiosk coordinates are not present, check the set difference between the kiosk names in the heartland_bikeshare and the kiosk names in the kiosk dataset.

In [13]:
import os
import dotenv
import pandas as pd
import numpy as np

dotenv.load_dotenv()

True

In [14]:
df_bike_trips = pd.read_parquet(os.path.join(os.environ['OUTPUT_PATH'], 'Cleaned_Bike_Trips.parquet'))

df_heartland_bike_share = pd.read_csv(os.path.join(os.environ['MOHAMMAD_SHARED_PATH'], "Backup","Heartland Bike Share - Kiosks.csv"), encoding='latin1')

In [15]:
df_heartland_bike_share = df_heartland_bike_share[['Kiosk Name', 'Address', 'Latitude', 'Longitude']]

# remove NA
df_heartland_bike_share = df_heartland_bike_share[df_heartland_bike_share['Latitude'].notna() & df_heartland_bike_share['Longitude'].notna()]
# remove lattitude and longitude that are 0
df_heartland_bike_share = df_heartland_bike_share[df_heartland_bike_share['Latitude'] != 0]
df_heartland_bike_share = df_heartland_bike_share[df_heartland_bike_share['Longitude'] != 0]

df_heartland_bike_share.sort_values(by=['Latitude', 'Longitude'], inplace=True)
df_heartland_bike_share['Coordinates'] = df_heartland_bike_share['Latitude'].astype(str) + ',' + df_heartland_bike_share['Longitude'].astype(str)
df_heartland_bike_share

,Kiosk Name,Address,Latitude,Longitude,Coordinates
93,Heartland Helpdesk,1144 N 11th St,39.97233,-75.14500,"39.97233,-75.145"
99,Mahoney State Park,Mahoney State Park,41.03038,-96.31184,"41.03038,-96.31184"
120,Walnut Creek Recreation Area (Papillion),11601 S 96th St,41.13997,-96.06433,"41.13997,-96.06433"
46,36th & Raynor Parkway (Bellevue),W Papio Trail,41.14896,-95.96814,"41.14896,-95.96814"
110,Prairie Queen Recreation Area (Papillion),Lincoln Rd,41.15487,-96.11240,"41.15487,-96.1124"
...,...,...,...,...,...
37,24th & Wirt St,3014 N 24th Street,41.28661,-95.94713,"41.28661,-95.94713"
104,NOTC 31st Ave & Taylor,4308 N 31st Ave,41.29800,-95.95830,"41.298,-95.9583"
100,MCC Fort Bookstore N 32nd St,N 32nd St,41.30424,-95.95923,"41.30424,-95.95923"
101,MCC North 30th St,N 30th St,41.30981,-95.95684,"41.30981,-95.95684"


#### Check if all the kiosk in bikeshare data have a valid coordinate in the heartland kiosks data

Since the set difference is not empty, we need to find if they are real kiosk or not and assign them their coordinates
    
- df_bike_trips - df_heartland_bike_share will give us the kiosk that are not in the heartland kiosk data

In [16]:
# we dont have the coordinates for these checkout kiosks
set(df_bike_trips['Checkout Kiosk'].unique()) - set(df_heartland_bike_share['Kiosk Name'].unique())

{'Bike Share HQ (Shop)',
 'Demo Station: 13th & Fahey',
 'Omaha’s Henry Doorly Zoo & Aquarium'}

In [17]:
#we dont have the coordinates for these return kiosks
set(df_bike_trips['Return Kiosk'].unique()) - set(df_heartland_bike_share['Kiosk Name'].unique())

{'(BTS) SLC Help Desk',
 'BCycle Hub',
 'BTS Customer Service Help Desk',
 'BTS Customer Service Help Desk (Virtual Kiosk)',
 'BTS Customer Service Helpdesk',
 'BTS Help Desk',
 'Bike Share HQ (Shop)',
 'Demo Station: 13th & Fahey',
 'Help Desk',
 'Helpdesk',
 'Houston BCycle Warehouse',
 'Omaha’s Henry Doorly Zoo & Aquarium',
 'RTC Helpdesk',
 "Ullman's",
 'Warehouse- Metro',
 'Z - BTS Customer Service Helpdesk',
 'Z - Help Desk (BTS Staff)',
 'Z - Virtual Kiosk (BTS STAFF) DO NOT USE JT'}

##### Instead of doing join and removing unneccessary columns, we will just add the coordinates to the bikeshare data by mapping the kiosk id to the coordinates

In [18]:
# make a dict of kioks and their coordinates
kiosk_coordinates = df_heartland_bike_share.set_index('Kiosk Name')['Coordinates'].to_dict()

In [19]:
df_bike_trips['Checkout_Kiosk_Coordinates'] = df_bike_trips['Checkout Kiosk'].copy().map(kiosk_coordinates)
df_bike_trips['Return_Kiosk_Coordinates'] = df_bike_trips['Return Kiosk'].copy().map(kiosk_coordinates)
df_bike_trips.sort_values(by=['Checkout_Kiosk_Coordinates', 'Return_Kiosk_Coordinates'], inplace=True)

In [20]:
# @ TODO Fix this, there should not be NA in coordinates

# remove the kioks that dont have coordinates
df_bike_trips = df_bike_trips[df_bike_trips['Checkout_Kiosk_Coordinates'].notna() & df_bike_trips['Return_Kiosk_Coordinates'].notna()]


In [21]:
df_bike_trips.to_parquet(os.path.join(os.environ['OUTPUT_PATH'], 'Bike_Trips_With_Coordinates.parquet'))